Allison Forte

DSC 540

Project: Final Milestone (5)

August 13, 2022

# Storing the data in a database, merging it, and visualizing it

Load your 3 cleaned and transformed datasets into a database. 
Load each dataset into SQL Lite as an individual table. 

In [2]:
# Install packages and establish a connection and the cursor

import sqlite3
con = sqlite3.connect('my_dsc540_database')
cur = con.cursor()

Create table with CSV file data

In [3]:
# pull in cleaned data from CSV from milestone 2

import numpy as np
import pandas as pd


def monthToNum(shortMonth):
    return {
            'Jan': 1,
            'Feb': 2,
            'Mar': 3,
            'Apr': 4,
            'May': 5,
            'Jun': 6,
            'Jul': 7,
            'Aug': 8,
            'Sep': 9, 
            'Oct': 10,
            'Nov': 11,
            'Dec': 12}[shortMonth]


file = pd.read_csv('/Users/allison.forte/Documents/540 Assignments/tech_fundings.csv')
df = pd.DataFrame(data=file)  # Create df
df.rename(columns = {'Region':'Country', 'Vertical':'Category', 'Funding Amount (USD)':'Funding_amt', 
                     'Funding Date':'Funding_date'}, inplace = True)  # Rename columns
df = df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)  # Drop missing data
df['Funding_amt'] = df['Funding_amt'].replace(['Unknown'],'0')  # Fix missing data
df = df.drop_duplicates(subset='Website', keep='first', inplace=False)  # Drop duplicates
df[['Funding_month', 'Funding_year']] = df['Funding_date'].str.split('-', expand=True)  # Split date to 2 columns
months = (df['Funding_month'])
month_num  = []
for m in months:
    month_num.append(monthToNum(m))
df['month_num'] = month_num  # Add month number to column
df['Funding_amt'] = df['Funding_amt'].astype(float)  # Change data type
df['Funding_year'] = df['Funding_year'].astype(float)  # Change data type
df = df[(df['Funding_amt'] <70000000) & (df['Funding_amt'] > 0)]  # Remove outliers
df['Country'] = df['Country'].str.lower()


# insert df into my_dsc540_database

df.to_sql(name = 'csv_data', con = con, if_exists = 'replace', index = False )


# commit the changes

con.commit()

/Users/allison.forte/opt/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


Create table with the website data

In [4]:
# pull in cleaned data from website from milestone 3

import lxml
import html5lib
import pandas as pd


url1 = 'https://www.iban.com/exchange-rates'
dfs1 = pd.read_html(url1)
df_convert = dfs1[0]

# Add one row with EUR conversion rate since the table is based on a EUR conversion rate
new_row = {'Currency':'EUR', 'Currency Name': 'Euro', 'Exchange Rate = 1 EUR':1, 'Convert':'NaN'}
df_convert = df_convert.append(new_row, ignore_index=True)

# Second source needed for the exchange rates- original data contained 'number' that was not the exchange rate
url2 = 'https://www.iban.com/currency-codes'
dfs2 = pd.read_html(url2)
df_codes = dfs2[0]

df_convert.rename(columns = {'Currency':'Currency_code', 'Currency Name':'Currency_name', 
                             'Exchange Rate = 1 EUR':'Exchange_rate_1EUR'}, inplace = True) # Change headers in df_convert
df_codes.rename(columns = {'Currency':'Currency_name', 'Code':'Currency_code'}, inplace = True) # Change headers in df_codes
df_convert.drop(columns = ['Convert'], inplace = True)  # Drop unneeded rows from df_convert
df_codes.drop(columns=['Number'], inplace = True)  # Drop unneeded rows from df_codes
df_codes.dropna(axis = 0, how = 'any', thresh = None, subset = None, inplace = True)  # Fixing missing data in df_codes
final_df = df_codes.merge(df_convert, on = 'Currency_code', suffixes=('', '_drop'))  # Merge the 2 data sets
final_df.drop(columns=['Currency_name_drop'], inplace = True)  # Drop the second currency_name column
final_df['Country'] = final_df['Country'].str.lower()
final_df


# insert final_df into my_dsc540_database

final_df.to_sql(name = 'website_data', con = con, if_exists = 'replace', index = False )


# commit the changes

con.commit()

Create table with the API data

In [5]:
# pull in cleaned data from API from milestone 4

import urllib.request, urllib.parse, urllib.error
import json
import requests


with open('/Users/allison.forte/Downloads/API keys') as f:  # Set up connection to API using API key

    api_keys = f.readlines()
    api_key = api_keys[9]


def search_currency(currency):
    try:
        code = currency
        url = 'https://v6.exchangerate-api.com/v6/' + api_key + 'latest/' + code
        print(f'Retrieving data for "{currency}".')  # Let the user know the search is working
        response = requests.get(url)  # Accessing the API
        data = response.json()  # Data received from the API

        if data['result'] == 'success':
            return(data)
    
        else:
            print("Error encountered")  # Informs user of error
    
    except urllib.error.URLError as e:
        print(f"ERROR: {e.reason}")  # Displays error if the request is not successful


code = final_df['Currency_code']  #  Pull the currency codes only and remove duplicates
codes = code.drop_duplicates(keep='first', inplace=False)

responses = [] 

for c in codes:
    responses.append(search_currency(c))
        
rates = {}
p = 0

for r in responses:
    rates[p] = r['conversion_rates']
    p=p+1
        
rates_df = pd.DataFrame.from_dict(rates, orient = 'columns')

names = {0: 'EUR', 1: 'USD', 2: 'AUD', 3:'INR', 4:'NOK', 5:'BRL', 6: 'BGN', 7: 'CAD', 8: 'CNY',
         9: 'NZD', 10: 'HRK', 11: 'CZK', 12: 'DKK', 13: 'GBP', 14: 'HKD', 15: 'HUF', 16: 'ISK',
         17: 'IDR', 18: 'ILS', 19: 'JPY', 20: 'KRW', 21: 'ZAR', 22: 'CHF', 23: 'MYR', 24: 'MXN', 
         25: 'PHP', 26: 'PLN', 27: 'RON', 28: 'SGD', 29: 'SEK', 30: 'THB', 31: 'TRY'}
rates_df.rename(columns=names, inplace = True)


# insert rates_df into my_dsc540_database

rates_df.to_sql(name = 'api_data', con = con, if_exists = 'replace', index = True )


# commit the changes

con.commit()

Retrieving data for "EUR".
Retrieving data for "USD".
Retrieving data for "AUD".
Retrieving data for "INR".
Retrieving data for "NOK".
Retrieving data for "BRL".
Retrieving data for "BGN".
Retrieving data for "CAD".
Retrieving data for "CNY".
Retrieving data for "NZD".
Retrieving data for "HRK".
Retrieving data for "CZK".
Retrieving data for "DKK".
Retrieving data for "GBP".
Retrieving data for "HKD".
Retrieving data for "HUF".
Retrieving data for "ISK".
Retrieving data for "IDR".
Retrieving data for "ILS".
Retrieving data for "JPY".
Retrieving data for "KRW".
Retrieving data for "ZAR".
Retrieving data for "CHF".
Retrieving data for "MYR".
Retrieving data for "MXN".
Retrieving data for "PHP".
Retrieving data for "PLN".
Retrieving data for "RON".
Retrieving data for "SGD".
Retrieving data for "SEK".
Retrieving data for "THB".
Retrieving data for "TRY".


Join the datasets together in Python into 1 dataset.

In [6]:
result = cur.execute("SELECT * FROM csv_data as csv \
                     LEFT JOIN website_data as web ON csv.Country = web.Country\
                     LEFT JOIN api_data as api")

Once all the data is merged together in your database, create 5 visualizations that demonstrate the data you have cleansed. 

# Write a 250-500-word summary of what you learned and had to do to complete the project. 

In your write-up, address the ethical implications of cleansing data and your project topic.

This project exposed me to processes that I was not familiar with. I had never pulled data from a website, and I had not used an API in this way. I appreciated the opportunity to explore these data sources and how to access them, clean them, and think through combining them with other data sources.  

I still have a lot to learn with APIs especially. I am getting more comfortable working with JSON data, but it still takes a lot to parse it and create something usable.  

For websites, I now know how to look at the source code and understand a bit more about the components of the website. This helps to understand exactly what I am trying to pull. While I will still need practice with this method of data extraction, the experience in this course seemed like a solid introduction to how most websites can be scrapped. 

I am most comfortable working with CSVs because I have the most experience with them. Starting with the CSV was helpful for me because I was able to look at the data and think about how I wanted to clean it which gave me ideas for cleaning the website and API data.  

One thing I was not prepared for was loading my data in SQLLite and then joining it. Doing a project like this in the future I would think about an entity relationship diagram early on so I could be sure my data would be easily matched. Learning about this in the last 2 weeks was extremely helpful and will allow me to be better prepared for projects in future classes.  

While I was unable to add visuals to this write-up, I do have some ethical concerns about where this project was heading. Looking at funding levels for companies in various countries could impact where investors are willing to put their money. If it were discovered that companies in specific countries tend to fund faster, perhaps investors would focus heavily on those countries while avoiding others. This data could also be combined with the first year's profits for a potentially problematic result. On the other hand, if the data could be used to discover which countries are able to make the most profits with the least funding capital, perhaps it would be worth further investigation into why that is to improve results in other countries.  

Data regarding finances across the world always has issues of equivalency. While this data was compiled to include exchange rates which should account for that, living conditions, lifestyle, and average income will always have an impact beyond what we can account for with a straightforward exchange rate. While this data could be a starting point, a lot more exploration could happen after an initial analysis. 